In [1]:
import os

os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/nishankode/datascienceproject.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = 'nishankode'
os.environ["MLFLOW_TRACKING_PASSWORD"] = '65b116e267585e4dd001984953c26fa5734c9a6a'

In [2]:
import os
print(os.getcwd())
os.chdir("../")
print(os.getcwd())

c:\Users\mnsnn\Documents\Portfolio\MLOps\datascienceproject\research
c:\Users\mnsnn\Documents\Portfolio\MLOps\datascienceproject


In [3]:
# Model Evaluation Entity

from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metrics_file_name: Path
    target_column: str
    mlflow_uri: str

In [4]:
# Updating Configuration Manager

from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories, save_json

class ConfigurationManager:
    def __init__(self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])
        
        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            test_data_path = config.test_data_path,
            model_path = config.model_path,
            all_params = params,
            metrics_file_name = config.metrics_file_name,
            target_column = schema.name,
            mlflow_uri = "https://dagshub.com/nishankode/datascienceproject.mlflow"
        )

        return model_evaluation_config


In [7]:
# Component ModelEvaluation

import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

class ModelEvaluation:

    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):

        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    def log_into_mlflow(self):

        # Loading Data and Model
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        # Splitting data
        test_x = test_data.drop(columns=[self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        # Configuring MLFLow
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_uri_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        # MLFlow Tracking
        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            rmse, mae, r2 = self.eval_metrics(test_y, predicted_qualities)

            # Saving metrics as local
            scores = {"rmse" : rmse, "mae" : mae, "r2" : r2}
            save_json(path = Path(self.config.metrics_file_name), data = scores)

            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)

            if tracking_uri_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticNetModelNew")
            else:
                mlflow.sklearn.log_model(model, "model")

In [8]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2025-07-27 14:59:03,780: INFO: common: YAML File: config\config.yaml Loaded Successfully.]
[2025-07-27 14:59:03,782: INFO: common: YAML File: params.yaml Loaded Successfully.]
[2025-07-27 14:59:03,787: INFO: common: YAML File: schema.yaml Loaded Successfully.]
[2025-07-27 14:59:03,789: INFO: common: Created Directory at artifacts]
[2025-07-27 14:59:03,792: INFO: common: Created Directory at artifacts/model_evaluation]
[2025-07-27 14:59:04,630: INFO: common: JSON file saved at artifacts\model_evaluation\metrics.json]


c:\Users\mnsnn\Documents\Portfolio\MLOps\datascienceproject\venv\lib\site-packages\_distutils_hack\__init__.py:15: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
c:\Users\mnsnn\Documents\Portfolio\MLOps\datascienceproject\venv\lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
Successfully registered model 'ElasticNetModelNew'.
2025/07/27 14:59:16 INFO 